In [ ]:
%%HTML
<link rel="stylesheet" type="text/css" href="../css/custom.css">

# LSTM


![footer_logo](../images/logo.png)

## Goal

We will discuss Long Short Term Memory Units (LSTMs) and how they better enable us to perform sequential tasks.

## Program

- [Recap: RNNs]()
- [The Vanishing/Exploding Gradients problem]()
- [LSTMs]()
- [GRU]()

# Recap: recurrent neural network (RNN)


- Proposed in the 80s for modeling time series

- An RNN does not start its "thinking" from scratch

- Networks can persist information

- Take earlier inputs into account when making predictions

![center three_quarters](../images/rnn/RNN_unrolled.png)


# Recap: RNN architectures

![center half](../images/rnn/rnn_sequence.jpeg)


# Recap: backpropagation through time ([BPTT](http://ir.hit.edu.cn/~jguo/docs/notes/bptt.pdf))

To train an RNN we backpropogate the error through time.

$$\frac{\partial L}{\partial w} = \Sigma_t\frac{\partial L_t}{\partial w}$$

Backpropagation of the $\delta$ error vectors through the network.

![center third](../images/rnn/bptt_recurrent.png)

# Vanishing gradients

> Long range dependencies are lost if there are many steps backwards through the network

![center third](../images/lstm/bptt_recurrent.png)


# Vanishing gradients: let's backpropagate

Let's define:

- Input to each recurrent time step $a_t = f(W_r\cdot a_{t-1}, W_h\cdot x_t)$. Where $f$ is an activation function.
- Output from each time step $\hat{y}_t = g(W_o\cdot a_t)$. Where $g$ is some function representing the ouput layer.
- The loss value on each time step $L_t = f(y_t, \hat{y}_t)$. Remember that the total loss in BPTT is defined as $L=\sum_t L_t$


<center><img src="../images/lstm/bptt_recurrent.png" width="400"><center>

# Vanishing gradients: let's backpropagate

![right third](../images/lstm/bptt_recurrent.png)

- $a_t = f(W_r\cdot a_{t-1}, W_h\cdot x_t)$
- $\hat{y}_t = g(W_o\cdot a_t)$
- $L_t = f(y_t, \hat{y}_t)$

*Back propagation through time:*

Let's focus on the loss at $t$, $L_t$.

For $W_o$ we have,

$\frac{\partial L_t}{\partial W_o} = \frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial W_o}$ 


# Vanishing gradients: let's backpropagate

![right third](../images/lstm/bptt_recurrent.png)

- $a_t = f(W_r\cdot a_{t-1}, W_h\cdot x_t)$
- $\hat{y}_t = g(W_o\cdot a_t)$
- $L_t = f(y_t, \hat{y}_t)$

*Back propagation through time:*

The weight $W_r$ is shared across all the time sequence. Therefore, we can differentiate to it at the each time step and sum all together:

$\frac{\partial L_t}{\partial W_r} = \sum_{n=0}^t\frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\frac{\partial a_t}{\partial a_n}\frac{\partial a_n}{\partial W_r}=\frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\sum_{n=0}^t\frac{\partial a_t}{\partial a_n}\frac{\partial a_n}{\partial W_r}$



# Vanishing gradients: let's backpropagate

![right third](../images/bptt_recurrent.png)

- $a_t = f(W_r\cdot a_{t-1}, W_h\cdot x_t)$
- $\hat{y}_t = g(W_o\cdot a_t)$
- $L_t = f(y_t, \hat{y}_t)$

*Back propagation through time:*

Using the chain rule and that $u_t = W_r \cdot a_t$, 

$\frac{\partial a_t}{\partial a_n}=\prod_{m=n+1}^t\frac{\partial a_m}{\partial a_{m-1}}=\prod_{m=n+1}^t\frac{\partial a_m}{\partial u_{m-1}}\frac{\partial u_{m-1}}{\partial W_r} =\prod_{m=n+1}^t\frac{\partial a_m}{\partial u_{m-1}}\cdot W_r$ 

So,

$\frac{\partial a_t}{\partial a_n}=W_r^{t - n}\prod_{m=n+1}^t f'(x)\bigg\rvert_{x=u_{m-1}}$ and therefore, $\frac{\partial L_t}{\partial W_r} = \frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\sum_{n=0}^tW_r^{t - n}\frac{\partial a_n}{\partial W_r}\prod_{m=n+1}^t f'(x)\bigg\rvert_{x=u_{m-1}}$



# Vanishing gradients: let's backpropagate

$$\frac{\partial L_t}{\partial W_r} = \frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\sum_{n=0}^tW_r^{t - n}\frac{\partial a_n}{\partial W_r}\prod_{m=n+1}^t f'(x)\bigg\rvert_{x=u_{m-1}}$$

Contribution to the loss coming from first observation $n=0$:

$$\frac{\partial L_t}{\partial W_r} = \Big(\color{red}{W_r^t}\cdot\prod_{m=1}^t f'(x)\bigg\rvert_{x=u_{m-1}}\Big)\cdot\frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\frac{\partial a_0}{\partial W_r}$$

- The term $\color{red}{W_r^t}$ can cause exploding and vanishing gradient problems. The contribution from observations explodes or decays exponentially as inputs are farther from the output ($t >> n$).


In [ ]:
0.01**100

In [ ]:
1.1**100

# Vanishing gradients: let's backpropagate

Contribution coming from first observation $n=0$:
$$\frac{\partial L_t}{\partial W_r} = \Big(W_r^t\cdot\color{red}{\prod_{m=1}^t f'(x)}\bigg\rvert_{x=u_{m-1}}\Big)\cdot\frac{\partial L_t}{\partial\hat{y}_t}\frac{\partial\hat{y}_t}{\partial a_t}\frac{\partial a_0}{\partial W_r}$$

- The term $\color{red}{\prod_{m=1}^t f'(x)}$ can cause vanishing gradients problems, since $f$ is an activation function

![center third](../images/lstm/vanishing_gradient_b.png)

<sup>See this [tutorial](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/) for a more details regarding the BPTT methodology.</sup>


# Vanishing & exploding gradients

Long range dependencies -> many steps backwards through the network -> vanishing/exploding gradients

This gradient issue with RNNs can be combatted by
- Weight regularization (exploding gradients due to $\color{red}{W_r^t}$)
- Weight clipping (vanishing/exploding gradients due to $\color{red}{W_r^t}$)
- ReLU activation functions (vanishing gradients due to $\color{red}{\prod_{m=1}^t f'(x)}$) 

However, alternative network architectures also address this issue.
- Long short term memory (LSTM)
- Gated recurrent unit (GRU)


# Long short term memory unit (LSTM)

- The concept of LSTMs was [first proposed in 1997](http://www.bioinf.jku.at/publications/older/2604.pdf) and nowadays they are frequently used in NLP.
- Popular approach to dealing with the vanishing gradient problem
- Efficiently learning long-range dependencies in sequences. 



# LSTM 

![center three_quarters](../images/lstm/LSTM_overview.png)



### LSTM cell memory

At the core of the LSTM cell is a state vector $\boldsymbol{C}_t$, it is the key to cell's 'memory'. In the figure below the continuity of the cell state vector is represented by the horizontal line running through the top of the diagram.


<center><img src="../images/lstm/LSTM_cell_state.png" width="500"><center>

Notice that the cell state runs straight down the entire chain, with only some minor linear interactions. As a result, it is very easy for information flow along it. This mechanism provides the network with the long term memory.


### Forgetting

The decision of what information to forget is made by the sigmoid 'forget gate layer', as shown in the figure below. This layer looks at the previous hidden state $\boldsymbol{h}_{t-1}$ and the current input $\boldsymbol{x}_t$. It outputs a vector $\boldsymbol{f}_t$ with numbers between $0$ and $1$, where $0$ represents *completely forgetting* the cell state value while $1$ represents *keep remembering* it.

![center half](../images/lstm/LSTM_forgetting.png)


### Storing

The decision of what new information to store is made by the interaction between the sigmoid 'input gate layer' and the tanh 'candidate state layer', as shown in the figure below. Both layers again look at the previous hidden state and the current input. The output vector of the input gate layer $\boldsymbol{i}_t$ modulates the candidate cell state $\tilde{\boldsymbol{C}}_t$ which is computed by the candidate state layer.

![center half](../images/lstm/LSTM_storing.png)



### Updating

Finally, the new cell state vector $\boldsymbol{C}_t$ is updated by applying the forget operation to the previous cell state and combining that with the modulated new candidate state. This process is detailed in the figure below.

![center three_quarters](../images/lstm/LSTM_updating.png)

### Generating output

Once the cell state vector has been computed the LSTM is ready to generate its output. The decision of what information to output is made by the sigmoid 'output gate layer', as shown in the figure below. Like the other gate layers, this also layer looks at previous hidden state and the current input and outputs a vector $\boldsymbol{o}_t$ with numbers between $0$ and $1$. The cell state is put through a $\tanh$ function to push its values between $-1$ and $1$, it is then modulated by the output gate to created the $\boldsymbol{h}_t$ hidden state vector.

![center three_quarters](../images/lstm/LSTM_output.png)


#### Question - Why "long short-term memory"?
Knowing how LSTMs work can you explain their name?

[LSTM paper](https://www.bioinf.jku.at/publications/older/2604.pdf)
>"Recurrent networks can in principle use their feedback connections to store representations of recent input events in the form of activations ("short-term memory", as opposed to "long-term memory embodied by slowly changing weights)"

## GRU, an alternative to LSTM

* The Gated Recurrent Unit is very similar to LSTM
* Similar optimization performance, however less computation overhead
* Notice how $z_i(t) \in [0, 1]$, being sigmoid selects the previous state, $h_i(t-1)$, or the candidate state $\hat{h}_i(t)$
* For more details, comparing LSTM and GRU, consult [this article](https://arxiv.org/abs/1412.3555)

![center half](../images/lstm/gru.png)

## GRU details

* When $z_i(t) = 0$:
    * Then $1 - z_i(t) = 1$, multiplication node does not affect the previous state: $h_i(t-1)*(1-z_i(t))$
    * Then $z_i(t) \hat{h}_i(t) = 0$. Addition node keeps hidden state unaffected
* When $z_i(t) = 1$:
    * Then $1- z_i(t) = 0$, mutliplication node clears the previous state
    * Then $z_i(t) \hat{h}_i(t) = \hat{h}_i(t)$. The addition node accepts the new state on the cleared state
        
        
![center half](../images/lstm/gru.png)        

## GRU vs. LSTM

- The key difference between GRU and LSTM is that GRU's bag has two gates that are reset and update while LSTM has three gates that are input, output, forget. GRU is less complex than LSTM because it has less number of gates.

- If the dataset is small then GRU is preferred otherwise LSTM for the larger dataset.

- GRU exposes the complete memory and hidden layers but LSTM doesn't.

# Summary

After reviewing the RNN we discussed its crucial flaw: the vanishing/exploding graident problem.

We then discussed solutions in the form of an LSTM and a GRU.

# LSTM Exercise
[Exercise: Human activity recognition](../exercises/03_02_lstm_human_activity_recognition.ipynb)

![footer_logo](../images/logo.png)